In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import sys
np.set_printoptions(formatter={'float': lambda x: format(x, '1.2E')})

In [2]:
nTurn = 1024
npt = 128

nCore_y = 2
nCore_z = 2

NL_c = 0.01
NL_t = 0.4
NL_nu= 0.3
NL_L = 1.8
dTune = 0.0

alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)

# 1. Preparation

In [3]:
beam, lattice = impact.readInputFile('test_toymodel.in');
beam.nCore_y = nCore_y
beam.nCore_z = nCore_z
beam.distribution.distribution_type = 'ReadFile'
beam.current = 0.0

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

reading ImpactZ input file (test_toymodel.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
def normalize(data):
    data[:,1] = (data[:,0]*alfx/np.sqrt(betx) + data[:,1]*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx/np.sqrt(betx) + data[:,3]*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    
def unnormalize(data):
    data[:,1] = (-data[:,0]*alfx + data[:,1])*NL_c/np.sqrt(betx)
    data[:,3] = (-data[:,2]*alfx + data[:,3])*NL_c/np.sqrt(betx)
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c

In [5]:
for i in range(len(lattice)-1,0,-1):
    if lattice[i].type == 'write_raw_ptcl':
        lattice.pop(i)
lattice[1].n_sckick = 100
lattice[1].length = NL_L
lattice[1].strength_t = NL_t
lattice[1].transverse_scale_c = NL_c
lattice[1].tune_advance = NL_nu

lattice[2].tune_advance = dTune
lattice[2].nonlinear_insert_tuneAdvance = NL_nu
lattice[2].nonlinear_insert_length = NL_L

In [6]:
lattice[0].turns = nTurn

TBT = impact.getElem('TBT')
TBT.file_id = nTurn
TBT.pID_begin = 1
TBT.pID_end = npt
lattice.insert(1,TBT)

# on X-axis

In [7]:
k=0
q_m = beam.multi_charge.q_m[0]
xmax = 0.98
X0 = np.arange(0.0,xmax+1.0e-15,xmax/(npt-1))
pTest = np.zeros([npt,9])
pTest[:,6] = q_m
pTest[:,-1] = np.arange(1,npt+1)
pTest[:,0] = copy(X0)
npt = len(pTest)
beam.n_particles = npt
print('initial npt=',npt)
unnormalize(pTest)

initial npt= 1


In [8]:
impact.writeParticleData(pTest, ke, mass, freq)
impact.writeInputFile(beam,lattice) 
impact.run(beam)

0

In [9]:
iTBT,TBT = impact.readTBT(nTurn,ke,mass,freq)
print(len(iTBT))

1


In [14]:
X = np.zeros([nTurn,npt],dtype=np.complex64)
for i in range(nTurn):
    normalize(TBT[i,:,:])
    X[i,:] = TBT[i,0,:] - 1j*TBT[i,1,:]

In [15]:
tuneX = np.zeros(npt)
for i in range(npt):
    signal = X[:,i] - X[:,i].mean()
    tuneX[i], amp, dummy = impact.util.naff(1,signal)

# on Y-axis

In [ ]:
k=0
q_m = beam.multi_charge.q_m[0]
ymax = 3.0
Y0 = np.arange(0.0,ymax+1.0e-15,ymax/(npt-1))
pTest = np.zeros([npt,9])
pTest[:,6] = q_m
pTest[:,-1] = np.arange(1,npt+1)
pTest[:,2] = copy(Y0)
npt = len(pTest)
beam.n_particles = npt
print('initial npt=',npt)
unnormalize(pTest)

In [ ]:
impact.writeParticleData(pTest, ke, mass, freq)
impact.run(beam)
iTBT,TBT = impact.readTBT(nTurn,ke,mass,freq)

In [ ]:
Y = np.zeros([nTurn,npt],dtype=np.complex64)
for i in range(nTurn):
    normalize(TBT[i,:,:])
    Y[i,:] = TBT[i,2,:] - 1j*TBT[i,3,:]

In [ ]:
tuneY = np.zeros(npt)
for i in range(npt):
    signal = Y[:,i] - Y[:,i].mean()
    tuneY[i], amp, dummy = impact.util.naff(1,signal)

# Plot

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(X0[1:-1],tuneX[1:-1]/0.3)
plt.xlabel('$x_n$');
plt.ylabel(r'$\omega_x$');
plt.subplot(1,2,2)
plt.plot(Y0[1:],tuneY[1:]/0.3)
plt.xlabel('$y_n$');
plt.ylabel(r'$\omega_y$');
plt.savefig('tune.onAxis.png',dpi=180)
plt.tight_layout()

In [ ]:
tuneY[1]/0.3

In [ ]:
tuneX[1]/0.3